In [ ]:
from knot_table_experiment import *

knot_table = retrieve_table()
while(True):
    k = generate_random_interesting_knot(10)
    table_lookup(k,knot_table)

100% [................................................................................] 2426 / 2426Took 982 to generate this one
New knot type: K3a1
Took 1413 to generate this one
Took 147 to generate this one
Took 174 to generate this one
Took 217 to generate this one
Took 24 to generate this one
Took 855 to generate this one
Took 434 to generate this one
